In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
 99% 1.05G/1.06G [00:10<00:00, 171MB/s]
100% 1.06G/1.06G [00:10<00:00, 113MB/s]


In [4]:
import zipfile
zip = zipfile.ZipFile('/content/dogs-vs-cats.zip' , 'r')
zip.extractall('/content/')
zip.close()

In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
batch_size = 16
# this is the augmentation configuration we will use for training
train_data = ImageDataGenerator(
rescale = 1./255,
shear_range = 0.2,
zoom_range = 0.2,
horizontal_flip = True)

#this is the augmentation configuration we will use for testing
#only scaling
test_data = ImageDataGenerator(rescale = 1./255)

In [10]:
#this is a generator that will read pictures found in subfolders of 'content/train'
# and idefinitely generate batches of augmented image data
train_generator = train_data.flow_from_directory(
'train',  # this is the target directory
target_size = (150, 150),  # all image will be resized to 150 X 150
batch_size = batch_size,
class_mode = 'binary')  # since we use binary_crossentropy loss , we need binary labels

# this is a similar generator , for validation data
validation_generator = test_data.flow_from_directory(
'test',
target_size = (150 , 150),
class_mode = 'binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D , Activation , Dropout , Flatten , Dense

In [13]:
model = Sequential()
model.add(Conv2D(32 , (3,3) , input_shape = (150 , 150 ,3)) )
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size= (2, 2)))

model.add(Conv2D(32 , (3,3) ))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size= (2, 2)))

model.add(Conv2D(32 , (3,3) ))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size= (2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation ('sigmoid'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [14]:
model.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

In [15]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)                    │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 148, 148, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 72, 72, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 72, 72, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 34, 34, 32)          │           9,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 34, 34, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 9248)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │         591,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              65 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_4 (Activation)            │ (None, 1)                   │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 611,393 (2.33 MB)

 Trainable params: 611,393 (2.33 MB)

 Non-trainable params: 0 (0.00 B)

In [21]:
model.fit(
train_generator ,
steps_per_epoch = 2000 // batch_size ,
epochs = 5,
validation_data = validation_generator ,
validation_steps = 800 // batch_size)

Epoch 1/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 546ms/step - accuracy: 0.5871 - loss: 0.6771

/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


125/125 ━━━━━━━━━━━━━━━━━━━━ 92s 729ms/step - accuracy: 0.5871 - loss: 0.6771 - val_accuracy: 0.6637 - val_loss: 0.6329
Epoch 2/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 87s 698ms/step - accuracy: 0.6498 - loss: 0.6516 - val_accuracy: 0.5938 - val_loss: 0.6714
Epoch 3/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 86s 688ms/step - accuracy: 0.6224 - loss: 0.6461 - val_accuracy: 0.7212 - val_loss: 0.5652
Epoch 4/5
125/125 ━━━━━━━━━━━━━━━━━━━━ 74s 593ms/step - accuracy: 0.6566 - loss: 0.6255 - val_accuracy: 0.7050 - val_loss: 0.5632
Epoch 5/5


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


125/125 ━━━━━━━━━━━━━━━━━━━━ 86s 692ms/step - accuracy: 0.6838 - loss: 0.5928 - val_accuracy: 0.7256 - val_loss: 0.5464
